In [42]:
from pathlib import Path
import pandas
import shutil
import pandas as pd
import os

### Read directories

In [43]:
output_dir = r'C:\DEER-SWHC024-05\commercial measures\SWHC024-05 Fan Belt\SWHC024-05 Fan Belt_New'

dir_result = os.path.join(output_dir, 'results-summary.csv')
dir_cohorts = os.path.join(output_dir, 'renamed_cohort.xlsx')

### extract multiple dataframes from results-summary.csv

In [44]:
def read_csv_with_multiple_dataframes(file_path):
    dataframes = []
    current_df = []
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  
            
            if not line:
                if current_df:  
                    df = pd.DataFrame(current_df[1:], columns=current_df[0])  # Use first row as column names
                    dataframes.append(df)
                    current_df = []  # Reset for the next DataFrame
            else:
                current_df.append(line.split(','))  
        
        if current_df:
            df = pd.DataFrame(current_df[1:], columns=current_df[0])  # Use first row as column names
            dataframes.append(df)
    
    return dataframes

file_path = dir_result
dataframes = read_csv_with_multiple_dataframes(file_path)


### change building prototype names to updated ones and save it as results-summary_new.csv 

In [45]:
cohort_df = pd.read_excel(dir_cohorts)

final_df = pd.DataFrame()

with open(output_dir + '\\results-summary_new.csv', 'w', newline='') as file:
    for i, result_df in enumerate(dataframes):
        result_df['Type'] = result_df['File Name'].str.split('/').str[1]
        result_df['Type'] = result_df['Type'].map(cohort_df.set_index('type')['revised cohort']).fillna(result_df['Type'])

        for index, row in result_df.iterrows():
            if pd.notna(row['File Name']):
                file_name_parts = row['File Name'].split('/')
                if len(file_name_parts) < 4:
                    continue  
                file_name_parts[1] = row['Type']
                new_file_name = '/'.join(file_name_parts)
                result_df.loc[index, 'File Name'] = new_file_name

        result_df.drop(columns=['Type'], inplace=True)
        
        result_df.to_csv(file, index=False)
        
        # Add an empty row between dataframes except for the last one
        if i < len(dataframes) - 1:
            file.write('\r\n')
